<a href="https://colab.research.google.com/github/Raph-09/Machine-Learning-Projects/blob/main/Ineuron_Project_(Musical_Instrument_dataset)Collaborative_Interactive_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import one_hot
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import StandardScaler
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Load data

In [6]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

m_df = getDF('/content/drive/MyDrive/Amazon core_5/meta_Musical_Instruments.json.gz')

In [7]:
m_df.head()

,asin,title,price,imUrl,salesRank,categories,related,brand,description
0,0006428320,"Six Sonatas For Two Flutes Or Violins, Volume ...",17.95,http://ecx.images-amazon.com/images/I/41EpRmh8...,{'Musical Instruments': 207315},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,NaN
1,0014072149,Double Concerto in D Minor By Johann Sebastian...,18.77,http://ecx.images-amazon.com/images/I/41m6ygCq...,{'Musical Instruments': 94593},[[Musical Instruments]],"{'also_viewed': ['B0058DK7RA'], 'buy_after_vie...",,Composer: J.S. Bach.Peters Edition.For two vio...
2,0041291905,Hal Leonard Vivaldi Four Seasons for Piano (Or...,NaN,http://ecx.images-amazon.com/images/I/41maAqSO...,{'Musical Instruments': 222972},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,Vivaldi's famous set of four violin concertos ...
3,0041913574,"Aida: Opera in Quattro Atti, Partitura -- Aida...",49.99,http://ecx.images-amazon.com/images/I/513kRMv%...,NaN,[[Musical Instruments]],NaN,NaN,444 pages. \nReprint of corrected and revised ...
4,0201891859,Nocturnes,NaN,http://ecx.images-amazon.com/images/I/41SXCAzs...,{'Musical Instruments': 171871},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,NaN


In [9]:
rev = pd.read_json('/content/drive/MyDrive/Amazon core_5/Musical_Instruments_5.json',lines=True)
rev.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


merge both meta data and review data

In [10]:
df = pd.merge(m_df,rev,on='asin')
df.head()

,asin,title,price,imUrl,salesRank,categories,related,brand,description,reviewerID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,1384719342,niceEshop Studio Microphone Mic Wind Screen Po...,7.09,http://ecx.images-amazon.com/images/I/510D2cVp...,{'Musical Instruments': 2880},"[[Musical Instruments, Microphones & Accessori...","{'also_bought': ['B009D015TI', 'B004QJREXM', '...",niceEshop,New generic Studio Microphone Mic Wind Screen ...,A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,1384719342,niceEshop Studio Microphone Mic Wind Screen Po...,7.09,http://ecx.images-amazon.com/images/I/510D2cVp...,{'Musical Instruments': 2880},"[[Musical Instruments, Microphones & Accessori...","{'also_bought': ['B009D015TI', 'B004QJREXM', '...",niceEshop,New generic Studio Microphone Mic Wind Screen ...,A14VAT5EAX3D9S,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,1384719342,niceEshop Studio Microphone Mic Wind Screen Po...,7.09,http://ecx.images-amazon.com/images/I/510D2cVp...,{'Musical Instruments': 2880},"[[Musical Instruments, Microphones & Accessori...","{'also_bought': ['B009D015TI', 'B004QJREXM', '...",niceEshop,New generic Studio Microphone Mic Wind Screen ...,A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,1384719342,niceEshop Studio Microphone Mic Wind Screen Po...,7.09,http://ecx.images-amazon.com/images/I/510D2cVp...,{'Musical Instruments': 2880},"[[Musical Instruments, Microphones & Accessori...","{'also_bought': ['B009D015TI', 'B004QJREXM', '...",niceEshop,New generic Studio Microphone Mic Wind Screen ...,A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,1384719342,niceEshop Studio Microphone Mic Wind Screen Po...,7.09,http://ecx.images-amazon.com/images/I/510D2cVp...,{'Musical Instruments': 2880},"[[Musical Instruments, Microphones & Accessori...","{'also_bought': ['B009D015TI', 'B004QJREXM', '...",niceEshop,New generic Studio Microphone Mic Wind Screen ...,A94QU4C90B1AX,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [18]:
#drop duplicate
df.drop_duplicates(subset='asin',inplace=True)

In [19]:
#total rows
len(df)

900

In [12]:
#columns data types
df.dtypes

asin               object
title              object
price             float64
imUrl              object
salesRank          object
categories         object
related            object
brand              object
description        object
reviewerID         object
reviewerName       object
helpful            object
reviewText         object
overall             int64
summary            object
unixReviewTime      int64
reviewTime         object
dtype: object

In [21]:
#check for missing values
df.isnull().sum()

asin                0
title               4
price               2
imUrl               2
salesRank           5
categories          0
related             2
brand             105
description         5
reviewerID          0
reviewerName       22
helpful             0
reviewText          0
overall             0
summary             0
unixReviewTime      0
reviewTime          0
dtype: int64

In [14]:
#fill na in title
df.title.isnull().sum()

24

In [27]:
#check rows with null values in title column
df[df['title'].isna()].head(10)

,asin,title,price,imUrl,salesRank,categories,related,brand,description,reviewerID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
2584,B0002GWXKC,NaN,NaN,NaN,NaN,[[Musical Instruments]],NaN,NaN,NaN,A1EFMEZJESPF76,"Ariana Rivadeneira ""Wari""","[2, 2]",I bought it by mistake cause I don't own nor p...,5,Fits well!!,1306972800,"06 2, 2011"
4174,B000AKSUP2,NaN,NaN,NaN,NaN,[[Musical Instruments]],NaN,NaN,NaN,A2Z1BOQPVQK2M4,Amazon Customer,"[0, 0]",Recently I switched from a Blackstar HT-5 to a...,5,If you don't have a tube amp... you need one o...,1389830400,"01 16, 2014"
4982,B000KY156E,NaN,149.99,http://ecx.images-amazon.com/images/I/51OzI30t...,{'Software': 2863},"[[Musical Instruments, Studio Recording Equipm...","{'also_bought': ['B00633YGKI', 'B000KY2JIM', '...",NaN,EZdrummer was designed to give you optimum dru...,AMTQJ1ZZ58BO4,Gordon Tubbs,"[1, 1]",Let me begin with a Public Service Announcemen...,4,"EZ, breezy, beautiful, drummer girl.",1381708800,"10 14, 2013"
9898,B009E3EWPI,NaN,49.57,http://ecx.images-amazon.com/images/I/51Fc9XDf...,{'Music': 42027},"[[CDs & Vinyl, Classic Rock], [CDs & Vinyl, Me...","{'also_bought': ['B0012YYZYK', 'B00008PX8P', '...",NaN,"On December 10, 2007, Led Zeppelin took the st...",A3HCK3UXD6WS4G,Carlos & Gloria Guerra,"[0, 0]",This music package by Led Zepplin is historica...,5,Great product,1394496000,"03 11, 2014"


In [36]:
#drop all null values 
df.dropna(inplace=True,subset=['title'])

In [37]:
df.isnull().sum()

asin                0
title               0
price               0
imUrl               0
salesRank           3
categories          0
related             0
brand             101
description         3
reviewerID          0
reviewerName       22
helpful             0
reviewText          0
overall             0
summary             0
unixReviewTime      0
reviewTime          0
dtype: int64

In [38]:
df.reset_index(inplace=True)

**Title**

In [40]:
#walking on the title column
#convert words into their stem and remove stopwords
s = PorterStemmer()
w_corpus = []
for i in range(0, len(df)):
    text = re.sub('[^a-zA-Z]', ' ', df['title'][i])#get only alphabets
    text = text.lower()
    text = text.split()
    
    text = [s.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    w_corpus.append(text)

In [41]:
w_corpus[1]

'monster standard instrument cabl straight straight quot plug'

In [42]:
#one hot encoding to get the index of the words
voc_size=6000
onehot_=[one_hot(words,voc_size)for words in w_corpus] 
print(onehot_[0])

[4692, 2907, 2522, 4681, 5436, 2862, 4578, 5624, 5144, 1423, 2089, 5187]


In [43]:
#add padding to the begining 
docs_emb=pad_sequences(onehot_,padding='pre',maxlen=500)
print(docs_emb[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [44]:
#use sequential model for the embedding
model=Sequential()
model.add(Embedding(voc_size,10,input_length=500))
model.compile('adam','mse')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 10)           60000     
Total params: 60,000
Trainable params: 60,000
Non-trainable params: 0
_________________________________________________________________


In [46]:
#get the word matrix
title_vectors = model.predict(docs_emb)

In [47]:
title_vectors[0]

array([[-0.03836219, -0.01161518, -0.00819027, ..., -0.00623165,
        -0.04223454, -0.0266343 ],
       [-0.03836219, -0.01161518, -0.00819027, ..., -0.00623165,
        -0.04223454, -0.0266343 ],
       [-0.03836219, -0.01161518, -0.00819027, ..., -0.00623165,
        -0.04223454, -0.0266343 ],
       ...,
       [-0.01140553,  0.01440168, -0.04881253, ...,  0.04409083,
        -0.00536419,  0.02356047],
       [-0.00538112, -0.02859413,  0.01709068, ...,  0.01016475,
         0.03822452, -0.00999828],
       [-0.04461968, -0.04070629,  0.0031907 , ...,  0.03080604,
         0.00396561, -0.0221677 ]], dtype=float32)

**Review Text**

In [49]:
#walking on the review text column
#convert words into their stem and remove stopwords
s = PorterStemmer()
w_corpus = []
for i in range(0, len(df)):
    text = re.sub('[^a-zA-Z]', ' ', df['reviewText'][i])#get only alphabets
    text = text.lower()
    text = text.split()
    
    text = [s.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    w_corpus.append(text)

In [50]:
#one hot encoding to get the index of the words
voc_size=6000
onehot_=[one_hot(words,voc_size)for words in w_corpus] 
#add padding to the begining 
docs_emb=pad_sequences(onehot_,padding='pre',maxlen=500)

In [51]:
#use sequential model for the embedding
model=Sequential()
model.add(Embedding(voc_size,10,input_length=500))
model.compile('adam','mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 10)           60000     
Total params: 60,000
Trainable params: 60,000
Non-trainable params: 0
_________________________________________________________________


In [53]:
#get the word matrix
reviewtext_vectors = model.predict(docs_emb)

In [54]:
reviewtext_vectors[0]

array([[ 0.0231103 , -0.00160276, -0.04089414, ...,  0.01032808,
        -0.01324234,  0.01073421],
       [ 0.0231103 , -0.00160276, -0.04089414, ...,  0.01032808,
        -0.01324234,  0.01073421],
       [ 0.0231103 , -0.00160276, -0.04089414, ...,  0.01032808,
        -0.01324234,  0.01073421],
       ...,
       [ 0.00625818,  0.01530762, -0.0465245 , ..., -0.03234117,
         0.03201996, -0.0202993 ],
       [ 0.00227784,  0.03560841, -0.03777625, ...,  0.02594162,
        -0.01775413,  0.02586317],
       [ 0.01934833,  0.04436921,  0.04535318, ..., -0.00818244,
        -0.01462258, -0.04385928]], dtype=float32)

**Summary**

In [57]:
#walking on the summry column
#convert words into their stem and remove stopwords
s = PorterStemmer()
w_corpus = []
for i in range(0, len(df)):
    text = re.sub('[^a-zA-Z]', ' ', df['summary'][i])#get only alphabets
    text = text.lower()
    text = text.split()
    
    text = [s.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    w_corpus.append(text)

In [58]:
#one hot encoding to get the index of the words
voc_size=6000
onehot_=[one_hot(words,voc_size)for words in w_corpus] 
#add padding to the begining 
docs_emb=pad_sequences(onehot_,padding='pre',maxlen=500)

In [59]:
#use sequential model for the embedding
model=Sequential()
model.add(Embedding(voc_size,10,input_length=500))
model.compile('adam','mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 10)           60000     
Total params: 60,000
Trainable params: 60,000
Non-trainable params: 0
_________________________________________________________________


In [62]:
#get the word matrix
summary_vectors = model.predict(docs_emb)

In [63]:
summary_vectors[0]

array([[ 0.03689387, -0.03391448, -0.00975294, ..., -0.00957855,
        -0.0230072 ,  0.01463487],
       [ 0.03689387, -0.03391448, -0.00975294, ..., -0.00957855,
        -0.0230072 ,  0.01463487],
       [ 0.03689387, -0.03391448, -0.00975294, ..., -0.00957855,
        -0.0230072 ,  0.01463487],
       ...,
       [ 0.03689387, -0.03391448, -0.00975294, ..., -0.00957855,
        -0.0230072 ,  0.01463487],
       [ 0.03689387, -0.03391448, -0.00975294, ..., -0.00957855,
        -0.0230072 ,  0.01463487],
       [ 0.03111989,  0.04624481, -0.04355292, ..., -0.03347212,
        -0.03395095,  0.01977307]], dtype=float32)

**Numerical Columns**


In [65]:
num_df = df[['price','overall','unixReviewTime']]

In [66]:
num_df.head()

,price,overall,unixReviewTime
0,7.09,5,1393545600
1,19.56,5,1356048000
2,12.55,4,1398124800
3,25.00,4,1376352000
4,6.79,4,1341964800


In [68]:
sc = StandardScaler()
sc.fit(num_df)
num_scaled = sc.transform(num_df)

In [69]:
num_scaled[0]

array([-0.56389629,  0.57398146,  0.86467196])

**Final matrix(Dataframe)**

In [70]:
#convert the scaled numerical data into dataframe
num_df = pd.DataFrame(num_scaled, columns=['price', 'overall','time(unix)'])

In [71]:
num_df.head()

,price,overall,time(unix)
0,-0.563896,0.573981,0.864672
1,-0.365502,0.573981,-0.076433
2,-0.477029,-0.568879,0.979600
3,-0.278953,-0.568879,0.433152
4,-0.568669,-0.568879,-0.429889


In [80]:
#add the title vectors to the table
vec_df = num_df.assign(title=[*title_vectors])

In [81]:
#add summary
vec_df_ = vec_df.assign(summary=[*summary_vectors])

In [84]:
#add reviewtext
matrix_df = vec_df_.assign(reviewtext=[*reviewtext_vectors])

In [85]:
matrix_df.head()

,price,overall,time(unix),title,summary,reviewtext
0,-0.563896,0.573981,0.864672,"[[0.023110297, -0.0016027577, -0.04089414, -0....","[[0.03689387, -0.033914484, -0.00975294, 0.023...","[[0.03689387, -0.033914484, -0.00975294, 0.023..."
1,-0.365502,0.573981,-0.076433,"[[0.023110297, -0.0016027577, -0.04089414, -0....","[[0.03689387, -0.033914484, -0.00975294, 0.023...","[[0.03689387, -0.033914484, -0.00975294, 0.023..."
2,-0.477029,-0.568879,0.979600,"[[0.023110297, -0.0016027577, -0.04089414, -0....","[[0.03689387, -0.033914484, -0.00975294, 0.023...","[[0.03689387, -0.033914484, -0.00975294, 0.023..."
3,-0.278953,-0.568879,0.433152,"[[0.023110297, -0.0016027577, -0.04089414, -0....","[[0.03689387, -0.033914484, -0.00975294, 0.023...","[[0.03689387, -0.033914484, -0.00975294, 0.023..."
4,-0.568669,-0.568879,-0.429889,"[[0.023110297, -0.0016027577, -0.04089414, -0....","[[0.03689387, -0.033914484, -0.00975294, 0.023...","[[0.03689387, -0.033914484, -0.00975294, 0.023..."
